In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Firstly, we will load the Dataset which have Image name and corresponding Labels

In [ ]:
Dataset = pd.read_csv("/kaggle/input/scene-classification/train-scene classification/train.csv")    # Dataframe

In [ ]:
# Converting Class number to strings
Dataset["label"] = Dataset["label"].astype(str)

In [ ]:
Dataset.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Displaying number of samples for each Disease
fig, ax = plt.subplots(figsize = (10, 4))                                # Setting Figure Size
sns.countplot(x ='label', data=Dataset)                                  # Creating Seaborn Count Plot
plt.xlabel("Class Label")                                                # X-Label of the plot
plt.ylabel("Number of Samples")                                          # Y-Label of the plot
plt.show() 

The Data is almost balanced.

## Splitting Data into Train/Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Data_train, Data_test = train_test_split(Dataset, test_size=0.2)                   # Splitting in 80:20

## Creating Keras Image Data Flow

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)                            # Normalizing pixels of images

## Creating Training Data

In [ ]:
# Training Set Directory
dir1='/kaggle/input/scene-classification/train-scene classification/train/'

In [ ]:
train_gen=datagen.flow_from_dataframe(dataframe = Data_train,           # Training Dataframe
                                      directory=dir1,                   # Training set Directory
                                      batch_size=20,                    # Size of Batch
                                      class_mode="categorical",         # Type of Labels
                                      x_col="image_name",               # Input Column
                                      color_mode="rgb",                 # Image Format
                                      y_col="label",                    # Target Column
                                      target_size=(224,224))            # Image Size

## Creating Validation Data

In [ ]:
valid_gen=datagen.flow_from_dataframe(dataframe = Data_test,            # Training Dataframe
                                      directory=dir1,                   # Training set Directory
                                      batch_size=20,                    # Size of Batch
                                      class_mode="categorical",         # Type of Labels
                                      x_col="image_name",               # Input Column
                                      color_mode="rgb",                 # Image Format
                                      y_col="label",                    # Target Column
                                      target_size=(224,224))            # Image Size

## Importing Resnet Model to apply Transfer Learning

In [ ]:
import keras

In [ ]:
ResNet_model = keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

## Building Model

In [ ]:
from keras import Model 
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from keras.models import Sequential

In [ ]:
for layer in ResNet_model.layers[:-15]:       # Freezing all layers other than last 15 Layers
    layer.trainable = False

x = ResNet_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=6, activation='softmax')(x)
model = Model(ResNet_model.input, output)

In [ ]:
# model Summary
print(model.summary())

### Setting Loss function, Optimizer and Compling the model

In [ ]:
loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])

### Compiling the Model

In [ ]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

In [ ]:
transfer_learning_history = model.fit_generator(generator=train_gen,
                            steps_per_epoch=STEP_SIZE_TRAIN,
                            validation_data=valid_gen,
                            validation_steps=STEP_SIZE_VALID,
                            epochs=3)

## Visualizing accuracy and loss

In [ ]:
import matplotlib.pyplot as plt

acc = transfer_learning_history.history['accuracy']
val_acc = transfer_learning_history.history['val_accuracy']

loss = transfer_learning_history.history['loss']
val_loss = transfer_learning_history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Since, we are using Pre-trained Moedl. The Model Convergers very fast.
# This is the reason we are getting best results in only 2 epochs